In [30]:
import datetime
test_start_date = datetime.datetime.now() - datetime.timedelta(days = 10)
test_end_date = datetime.datetime.now() #- datetime.timedelta(days = 1)


In [19]:
import numpy as np
import pandas as pd
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [20]:
# Reading the saved data pickle file
df_stocks = pd.read_pickle('pickled_para.pkl')
df_stocks

,close,articles
2018-03-03 02:30:00,24538.06,. Trump’s Yanked Support for Hudson Tunnel Ang...
2018-03-06 02:30:00,24874.76,. Cosby Lawyers: #MeToo Moment Makes a Fair Tr...
2018-03-07 02:30:00,24884.12,". Kushner Meets With Mexican President, Unders..."
2018-03-08 02:30:00,24801.36,". Deadly Storm Fells Trees, Causing Travel Woe..."
2018-03-09 02:30:00,24895.21,. Quebec Doctors Make a Pay Demand: Don’t Rais...
2018-03-10 02:30:00,25335.74,. Paralympic Skiers Confront the Scars That Li...
2018-03-13 01:30:00,25178.61,. Trump’s Killing of Chip Deal Pushes Protecti...
2018-03-14 01:30:00,25007.03,. Character Was Destiny for Spitzer; Will It B...
2018-03-15 01:30:00,24758.12,. French Judge Has Issued Arrest Warrant for S...
2018-03-16 01:30:00,24873.66,". James Levine, a Fractured Partnership and a ..."


In [21]:
df_stocks['prices'] = df_stocks['close'].apply(np.int64)
# selecting the prices and articles
df_stocks = df_stocks[['prices', 'articles']]

In [22]:
df_stocks['articles'] = df_stocks['articles'].map(lambda x: x.lstrip('.-'))

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
df_stocks
df = df_stocks[['prices']].copy()
df

,prices
2018-03-03 02:30:00,24538
2018-03-06 02:30:00,24874
2018-03-07 02:30:00,24884
2018-03-08 02:30:00,24801
2018-03-09 02:30:00,24895
2018-03-10 02:30:00,25335
2018-03-13 01:30:00,25178
2018-03-14 01:30:00,25007
2018-03-15 01:30:00,24758
2018-03-16 01:30:00,24873


In [24]:
# Adding new columns to the data frame
df["compound"] = ''
df["neg"] = ''
df["neu"] = ''
df["pos"] = ''
df

,prices,compound,neg,neu,pos
2018-03-03 02:30:00,24538,,,,
2018-03-06 02:30:00,24874,,,,
2018-03-07 02:30:00,24884,,,,
2018-03-08 02:30:00,24801,,,,
2018-03-09 02:30:00,24895,,,,
2018-03-10 02:30:00,25335,,,,
2018-03-13 01:30:00,25178,,,,
2018-03-14 01:30:00,25007,,,,
2018-03-15 01:30:00,24758,,,,
2018-03-16 01:30:00,24873,,,,


In [25]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata
sid = SentimentIntensityAnalyzer()
for date, row in df_stocks.T.iteritems():
    try:
        sentence = unicodedata.normalize('NFKD', df_stocks.loc[date, 'articles'])
        ss = sid.polarity_scores(sentence)
        df.set_value(date, 'compound', ss['compound'])
        df.set_value(date, 'neg', ss['neg'])
        df.set_value(date, 'neu', ss['neu'])
        df.set_value(date, 'pos', ss['pos'])
    except TypeError:
        print (df_stocks.loc[date, 'articles'])
        print (date)

In [26]:
df

,prices,compound,neg,neu,pos
2018-03-03 02:30:00,24538,-0.9979,0.183,0.711,0.106
2018-03-06 02:30:00,24874,-0.8381,0.101,0.807,0.092
2018-03-07 02:30:00,24884,0.9837,0.101,0.783,0.116
2018-03-08 02:30:00,24801,-0.9984,0.123,0.8,0.077
2018-03-09 02:30:00,24895,-0.998,0.136,0.763,0.101
2018-03-10 02:30:00,25335,-0.9656,0.123,0.783,0.094
2018-03-13 01:30:00,25178,-0.9902,0.116,0.79,0.095
2018-03-14 01:30:00,25007,-0.9982,0.167,0.704,0.129
2018-03-15 01:30:00,24758,-0.982,0.089,0.841,0.07
2018-03-16 01:30:00,24873,-0.9885,0.104,0.809,0.087


In [31]:
#import time
#import datetime


test = df.ix[test_start_date:test_end_date]


##
#test_start_date = '2018-03-09'
#test_end_date = '2018-03-10'
test = df.ix[test_start_date:test_end_date]
  
##    
    
# Calculating the sentiment score
sentiment_score_list = []
for date, row in test.T.iteritems():
    sentiment_score = np.asarray([df.loc[date, 'compound'],df.loc[date, 'neg'],df.loc[date, 'neu'],df.loc[date, 'pos']])
    #sentiment_score = np.asarray([df.loc[date, 'neg'],df.loc[date, 'pos']])
    sentiment_score_list.append(sentiment_score)
    numpy_df_test = np.asarray(sentiment_score_list)
      

In [32]:
numpy_df_test

array([[-0.998 ,  0.119 ,  0.806 ,  0.074 ],
       [ 0.8211,  0.099 ,  0.796 ,  0.105 ],
       [-0.9934,  0.126 ,  0.769 ,  0.106 ],
       [-0.9884,  0.108 ,  0.8   ,  0.091 ],
       [-0.8397,  0.106 ,  0.798 ,  0.096 ],
       [ 0.    ,  0.    ,  0.    ,  0.    ]])

In [33]:
import pickle

filename = 'finalized_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
#loaded_model.fit(numpy_df_test,test['prices'])   
result = loaded_model.predict(numpy_df_test)

difference=result[0]-result[1]
if(difference>0):
    sentence="Stock Price will rise"
else:
    sentence="Stock Price will fall"

print(result)
print(difference)
print(sentence)

[18533 18066 18552 16086 17986 18503]
467
Stock Price will rise
